# pytesseract --help-oem

In [ ]:
# import imutils
from imutils.object_detection import non_max_suppression
import numpy as np
import pytesseract
import argparse
import cv2

In [ ]:
def decode_predictions(scores, geometry):
    (numRows, numCols) = scores.shape[2:4]
    rects= []
    confidences = []
    
    for y in range(0, numRows):
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]
        
        for x in range(0, numCols):
            if scoresData[x] < args["min_confidence"]:
                continue
 
            (offsetX, offsetY) = (x * 4.0, y * 4.0)
 
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
 
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
 
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)
 
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])
 
    return (rects, confidences)

In [ ]:
!pip install easydict

import  easydict 

In [ ]:
# construct the argument parser and parse the arguments

parser = easydict.EasyDict()
parser.add_argument("--image", type=str, help="path to input image")
parser.add_argument("--east", type=str, help="path to input EAST text detector")
parser.add_argument("--min", type=float, default=0.5, help="minimum probability required to inspect a region")
parser.add_argument("--width", type=int, default=320, help="nearest multiple of 32 for resized width")
parser.add_argument("--height", type=int, default=320, help="nearest multiple of 32 for resized height")
parser.add_argument("--padding", type=float, default=0.0, help="amount of padding to add to each border of ROI")

# parser.add_argument("path to input image")
# parser.add_argument("path to input EAST text detector")
# parser.add_argument("minimum probability required to inspect a region")
# parser.add_argument("nearest multiple of 32 for resized width")
# parser.add_argument("nearest multiple of 32 for resized height")
# parser.add_argument("amount of padding to add to each border of ROI")

args = parser.parse_args()

In [ ]:
# load the input image and grab the image dimensions
image = cv2.imread(args["image"])
orig = image.copy()
(origH, origW) = image.shape[:2]

# set the new width and height and then determine the ratio in change
# for both the width and height
(newW, newH) = (args["width"], args["height"])
rW = origW / float(newW)
rH = origH / float(newH)

# resize the image and grab the new image dimensions
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]

In [ ]:
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]

# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(args["east"])

In [ ]:
# construct a blob from the image and then perform a forward pass of
# the model to obtain the two output layer sets
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
    (123.68, 116.78, 103.94), swapRB=True, crop=False)
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)

# decode the predictions, then  apply non-maxima suppression to
# suppress weak, overlapping bounding boxes
(rects, confidences) = decode_predictions(scores, geometry)
boxes = non_max_suppression(np.array(rects), probs=confidences)

In [ ]:
# initialize the list of results
results = []

# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
	# scale the bounding box coordinates based on the respective
	# ratios
	startX = int(startX * rW)
	startY = int(startY * rH)
	endX = int(endX * rW)
	endY = int(endY * rH)

	# in order to obtain a better OCR of the text we can potentially
	# apply a bit of padding surrounding the bounding box -- here we
	# are computing the deltas in both the x and y directions
	dX = int((endX - startX) * args["padding"])
	dY = int((endY - startY) * args["padding"])

	# apply padding to each side of the bounding box, respectively
	startX = max(0, startX - dX)
	startY = max(0, startY - dY)
	endX = min(origW, endX + (dX * 2))
	endY = min(origH, endY + (dY * 2))

	# extract the actual padded ROI
	roi = orig[startY:endY, startX:endX]

In [ ]:
	config = ("-l eng --oem 1 --psm 7")
	text = pytesseract.image_to_string(roi, config=config)
 
	# add the bounding box coordinates and OCR'd text to the list
	# of results
	results.append(((startX, startY, endX, endY), text))

In [ ]:
# sort the results bounding box coordinates from top to bottom
results = sorted(results, key=lambda r:r[0][1])

# loop over the results
for ((startX, startY, endX, endY), text) in results:
	# display the text OCR'd by Tesseract
	print("OCR TEXT")
	print("========")
	print("{}\n".format(text))

	# strip out non-ASCII text so we can draw the text on the image
	# using OpenCV, then draw the text and a bounding box surrounding
	# the text region of the input image
	text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
	output = orig.copy()
	cv2.rectangle(output, (startX, startY), (endX, endY),
		(0, 0, 255), 2)
	cv2.putText(output, text, (startX, startY - 20),
		cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

	# show the output image
	cv2.imshow("Text Detection", output)
	cv2.waitKey(0)

# tesseract --help-psm

In [ ]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import image_to_string



# import tensorflow as tf
# import tensorflow 

# from tensorflow import keras
# from keras.layers import Dense

# from keras.models import load_model
# import sys

In [ ]:
src_patch = "images/" 

In [ ]:
def get_string(img_path):
    img = cv2.imread(img_path)  # Lendo imagem com OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convertendo pra cinza

    # Aplicando dilatação e erosão  para remover alguns ruidos
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.erode(img, kernel, iterations=1)
    img = cv2.dilate(img, kernel, iterations=1)
    
    # img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    # img = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)
    # img = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)
    
    img = cv2.Sobel(img,cv2.CV_8U,1,0,ksize=5)
    

    cv2.imwrite("removed_noise.png", img)  # Sobreescrevendo a imagem depois de remover ruido

    #  Apply threshold to get image with only black and white
    #img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    cv2.imwrite("thres.png", img)  # Escrevendo a imagem depois de ser processada pelo OpenCV, para fazer algo..

    result = pytesseract.image_to_string(Image.open("thres.png"))  # Reconhecendo texto com Teeserract

    # Remove template file
    #os.remove(temp)

    return result

In [ ]:
print('-- Iniciando recohecimento de Imagem --')
print(get_string("pt.jpg"))
print('------ Feito ------')

In [ ]:
def get_string(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.erode(img, kernel, iterations=1)
    
    #dst = cv2.cornerHarris(kernel,3,3,0.00001)
    img = cv2.Canny(img, threshold1=200, threshold2=300)
    img = cv2.GaussianBlur(img, (3,3), 0 )

    
    cv2.imwrite("removed_noise.png", img)
    cv2.imwrite("thres.png", img)
    result = pytesseract.image_to_string(Image.open("thres.png"))
    
    return result

In [ ]:
print('-- Iniciando recohecimento de Imagem --')
print(get_string("land03.png"))
print('------ Feito ------')

In [ ]:
from skimage import io
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [ ]:
def predict(url):
    global model      
    # Read image
    image = io.imread(url)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (500, 500), interpolation=cv2.INTER_CUBIC)    

    # Use otsu to mask
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    mask = cv2.medianBlur(mask, 5)

    img = mask
    
    '''
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.erode(img, kernel, iterations=1)
    
    #dst = cv2.cornerHarris(kernel,3,3,0.00001)
    img = cv2.Canny(img, threshold1=200, threshold2=300)
    img = cv2.GaussianBlur(img, (3,3), 0 )
    '''
    
    
    cv2.imwrite("removed_noise.png", img)
    cv2.imwrite("thres.png", img)
    result = pytesseract.image_to_string(Image.open("thres.png"))
    
    return result

In [ ]:
print('-- Iniciando recohecimento de Imagem --')
print(predict("land03.png"))
print('------ Feito ------')

# contors

In [ ]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import image_to_string

import imutils
from imutils import contours
import argparse

In [ ]:
def test(img_path):
    
    img = io.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.threshold(img, 10, 255, cv2.THRESH_BINARY_INV)[1]
    
    refCnts = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    refCnts = imutils.grab_contours(refCnts)
    refCnts = contours.sort_contours(refCnts, method="left-to-right")[0]
    
    img = imutils.resize(img, width=300)
    kernel = np.ones((1, 1), np.uint8)

    tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, kernel)


    cv2.imwrite("removed_noise.png", img)
    cv2.imwrite("thres.png", img)
    result = pytesseract.image_to_string(Image.open("thres.png"))
    
    return result

In [ ]:
print('-- Iniciando recohecimento de Imagem --')
print(test("panelaaco.png"))
print('------ Feito ------')

In [ ]:
import cv2
import numpy as np

img = cv2.imread('number.jpg')

# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED) 

In [ ]:
image = cv2.imread('land03.png')

gray = get_grayscale(image)
thresh = thresholding(gray)
opening = opening(gray)
canny = canny(gray)

# scom

In [ ]:
import sys
from skimage import io
import numpy as np
import cv2

im = cv2.imread('land03.jpg')
a=[[5,2],[4,3]]
c=np.asarray(im,dtype=float)
im3 = c.copy()

gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray,(5,5),0)
thresh = cv2.adaptiveThreshold(blur,255,1,1,11,2)

#################      Now finding Contours         ###################

contours,hierarchy = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

samples =  np.empty((0,100))
responses = []
keys = [i for i in range(48,58)]

for cnt in contours:
    if cv2.contourArea(cnt)>50:
        [x,y,w,h] = cv2.boundingRect(cnt)

        if  h>28:
            cv2.rectangle(im,(x,y),(x+w,y+h),(0,0,255),2)
            roi = thresh[y:y+h,x:x+w]
            roismall = cv2.resize(roi,(10,10))
            cv2.imshow('norm',im)
            key = cv2.waitKey(0)

            if key == 27:  # (escape to quit)
                sys.exit()
            elif key in keys:
                responses.append(int(chr(key)))
                sample = roismall.reshape((1,100))
                samples = np.append(samples,sample,0)

responses = np.array(responses,np.float32)
responses = responses.reshape((responses.size,1))
print("training complete")

np.savetxt('generalsamples.data',samples)
np.savetxt('generalresponses.data',responses)

In [ ]:
import pytesseract as ocr
import numpy as np
import cv2

from PIL import Image

imagem = Image.open('panela.jpg').convert('RGB')

npimagem = np.asarray(imagem).astype(np.uint8)  

# diminuição dos ruidos antes da binarização
npimagem[:, :, 2] = 0 # zerando o canal R (RED)
npimagem[:, :, 0] = 0 # zerando o canal B (BLUE)

im = cv2.cvtColor(npimagem, cv2.COLOR_RGB2GRAY) 

ret, thresh = cv2.threshold(im, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 

binimagem = Image.fromarray(thresh) 

phrase = ocr.image_to_string(binimagem, lang='eng')

print(phrase) 

In [ ]:
def final(img_path):
    img = Image.open(img_path).convert('RGB')
    
    npimagem = np.asarray(img).astype(np.uint8)  
    npimagem[:, :, 0] = 0 # zerando o canal R (RED)
    npimagem[:, :, 2] = 0 # zerando o canal B (BLUE)

    im = cv2.cvtColor(npimagem, cv2.COLOR_BGR2GRAY)
    
    ret, img = cv2.threshold(im, 255, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
    #img = Image.fromarray(thresh) 
    #img = ocr.image_to_string(binimagem, lang='eng')
    

    cv2.imwrite("removed_noise.png", img)
    cv2.imwrite("thres.png", img)
    result = pytesseract.image_to_string(Image.open("thres.png"))

    return result


In [ ]:
print('-- Iniciando recohecimento de Imagem --')
print(final("land01.png"))
print('------ Feito ------')

#  pyautogui

In [ ]:
import numpy as np
import pytesseract
import argparse
from PIL import Image
import cv2
import sys
from skimage import io
import imutils
from imutils import contours
import argparse
import pyautogui
import seaborn as sns

In [ ]:
def get_text(image):
    return pytesseract.image_to_string(image)

#answer2 = pyautogui.screenshot("land02 copy.png",region=(1526, 1142, 800, 800))
img = Image.open("land02 copy.png")
answer2 = get_text(img)

answer2

In [ ]:

from PIL import Image
import pytesseract

img = Image.open("land03.png")

print(pytesseract.image_to_string(img))

In [ ]:
import cv2
img = cv2.imread('land02 copy.png')

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img)
plt.axis('off')

In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
r = img.copy()
r = r[:, :, 0]
g = img.copy()
g = g[:, :, 1]
b = img.copy()
b = b[:, :, 2]

In [ ]:
plt.imshow(b,cmap='Blues_r')

In [ ]:
plt.imshow(r,cmap='Reds_r')

In [ ]:
plt.imshow(g,cmap='Greens_r')

In [ ]:
sns.distplot(img.flatten())
plt.xlim([0,256])

In [ ]:
sns.kdeplot(r.flatten(),color='red',bw=0.005)
sns.kdeplot(g.flatten(),color='green',bw=0.005)
sns.kdeplot(b.flatten(),color='blue',bw=0.005)
plt.xlim([0,256])

In [ ]:
kernel = np.array([[0 , -1,  0],
                   [-1,  5, -1],
                   [0 , -1,  0]])

opencvSaida= cv2.filter2D(img, -1, kernel)

plt.imshow(opencvSaida)

In [ ]:
kernel = np.array([[-1, -1, -1],
                   [-1,  8, -1],
                   [-1, -1, -1]])

opencvSaida = cv2.filter2D(img, -1, kernel)

plt.imshow(opencvSaida)

In [ ]:
kernel = np.array([[ 1,  1,  1],
                   [ 0,  0,  0],
                   [-1, -1, -1]])

opencvSaida = cv2.filter2D(img, -1, kernel)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.GaussianBlur(img,(21,21),0)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.imshow(opencvSaida)

In [ ]:
kernel = np.ones((1, 1), np.uint8)
tt = cv2.erode(img, kernel, iterations=1)
tt = cv2.dilate(img, kernel, iterations=1)

opencvSaida = cv2.filter2D(tt, -1, kernel)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.Sobel(img,cv2.CV_8U,1,0,ksize=5)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.erode(img, kernel, iterations=1)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.cornerHarris( kernel,3,3,0.00001)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.Canny(img, threshold1=200, threshold2=300)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.GaussianBlur(img, (3,3), 0 )

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
opencvSaida = cv2.resize(img, (500, 500), interpolation=cv2.INTER_CUBIC)

plt.imshow(opencvSaida)

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, opencvSaida = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
opencvSaida = cv2.medianBlur(opencvSaida, 5)

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
opencvSaida = cv2.threshold(img, 10, 255, cv2.THRESH_BINARY_INV)[1]

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
opencvSaida = imutils.grab_contours(opencvSaida)
opencvSaida = contours.sort_contours(opencvSaida, method="left-to-right")[0]

plt.imshow(opencvSaida)

In [ ]:
opencvSaida = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.imshow(opencvSaida) # get_grayscale

In [ ]:
opencvSaida = cv2.medianBlur(img,5)

plt.imshow(opencvSaida) # remove_noise

In [ ]:
opencvSaida = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

plt.imshow(opencvSaida) # thresholding

In [ ]:
kernkel_dilation = np.ones((5,5),np.uint8)
opencvSaida = cv2.dilate(img, kernkel_dilation, iterations = 1)

plt.imshow(opencvSaida) # thresholding

In [ ]:
kernel_erosion = np.ones((5,5),np.uint8)
opencvSaida = cv2.erode(img, kernel_erosion, iterations = 1)

plt.imshow(opencvSaida) # erosion

In [ ]:
kernel_opening = np.ones((5,5),np.uint8)
opencvSaida = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel_opening)

plt.imshow(opencvSaida) # opening - erosion followed by dilation

In [ ]:
opencvSaida = cv2.Canny(img, 100, 200)

plt.imshow(opencvSaida) # canny edge detection

In [ ]:
coords = np.column_stack(np.where(img > 0))
angle = cv2.minAreaRect(coords)[-1]
if angle < -45:
    angle = -(90 + angle)
else:
    angle = -angle
(h, w) = image.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
opencvSaida = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

plt.imshow(opencvSaida) # skew correction

In [ ]:
template = cv2.imread('land03.png',0)
opencvSaida = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED) 

plt.imshow(opencvSaida) # template matching

In [ ]:
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray,(5,5),0)
thresh = cv2.adaptiveThreshold(blur,255,1,1,11,2)
contours,hierarchy = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
opencvSaida  

plt.imshow(hierarchy)

In [ ]:
template = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
ret, thresh = cv2.threshold(template, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
opencvSaida

plt.imshow(thresh)